In [1]:
import re
import pandas as pd
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn

In [2]:
df = pd.read_csv("./merged_climate_emergency.csv")
df.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (2,8,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,...,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,truncated,user,withheld_in_countries
0,0,NaN,NaN,2020-03-06 17:22:02+00:00,"{'hashtags': [{'indices': [64, 72], 'text': 'B...",NaN,0,False,NaN,1235978963656364032,...,NaN,NaN,12,False,"{'contributors': None, 'coordinates': None, 'c...","<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @carla_denyer: This week has been a particu...,False,"{'contributors_enabled': False, 'created_at': ...",NaN
1,1,NaN,NaN,2020-03-06 17:22:01+00:00,"{'hashtags': [{'indices': [35, 52], 'text': 'C...",NaN,0,False,NaN,1235978956937089032,...,NaN,NaN,12,False,"{'contributors': None, 'coordinates': None, 'c...","<a href=""https://buffer.com"" rel=""nofollow"">Bu...",RT @foeeurope: The kids: this is a #ClimateEme...,False,"{'contributors_enabled': False, 'created_at': ...",NaN
2,2,NaN,NaN,2020-03-06 17:21:58+00:00,"{'hashtags': [{'indices': [106, 123], 'text': ...",NaN,0,False,NaN,1235978945797001216,...,NaN,NaN,2,False,"{'contributors': None, 'coordinates': None, 'c...","<a href=""https://about.twitter.com/products/tw...",RT @dpcarrington: Just how wet and warm the UK...,False,"{'contributors_enabled': False, 'created_at': ...",NaN
3,3,NaN,NaN,2020-03-06 17:21:57+00:00,"{'hashtags': [], 'symbols': [], 'urls': [], 'u...",NaN,0,False,NaN,1235978939262267394,...,1.235602e+18,1.235602e+18,6,False,"{'contributors': None, 'coordinates': None, 'c...","<a href=""http://twitter.com/download/android"" ...",RT @Dan1elGallardo: Ojalá @LaLiga apoye a @gre...,False,"{'contributors_enabled': False, 'created_at': ...",NaN
4,4,NaN,NaN,2020-03-06 17:21:47+00:00,"{'hashtags': [{'indices': [47, 61], 'text': 'C...",NaN,0,False,NaN,1235978899919638529,...,1.235888e+18,1.235888e+18,3,False,"{'contributors': None, 'coordinates': None, 'c...","<a href=""https://Eco1stArt.com"" rel=""nofollow""...",RT @Green_DannyB: Its worth restating that the...,False,"{'contributors_enabled': False, 'created_at': ...",NaN


In [3]:
df.shape

(89350, 32)

In [4]:
from nltk.corpus import stopwords

In [5]:
NEGATE = {"aint", "arent", "cannot", "cant", "couldnt", "didnt", "doesnt", "ain't", "aren't", "can't", "couldn't",  "didn't", "doesn't", "dont", "hadnt", "hasnt", "havent", "isnt","neither", "don't", "hadn't", "hasn't", "haven't", "isn't","neednt", "needn't", "never", "none", "nope", "nor", "not", "nothing", "nowhere", "shouldnt", "wasnt", "werent","shouldn't", "wasn't", "weren't", "without", "wont", "wouldnt", "won't", "wouldn't"}


In [6]:
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', " ".join(EMOTICONS[emot].replace(",","").split()), text)
    return text

def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()))
    return text.replace("_", " ")

In [7]:
def data_clean(text):
   # text="RT @GretaThunberg: On March 13-14th there will be big mobilisations for the climate in France! I will join the climate strike in Grenoble o\u2026"
    text = text.strip().lower()
        # Remove hyperlinks
    text = re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?','', text)
    # Remove RT and UserName(@username)
    text = re.sub(r"[@]\w+[:]", "", re.sub(r"^ *rt +", "", text)).strip()
        # Emoticons conversion
    text = convert_emoticons(text)
        # Remove special character
    text = re.sub(r"[^A-Za-z0-9 ]+","", text)
        # EMOJI CONVERSION
    text = convert_emojis(text)
        #Stop Word Removal
    text = re.sub(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*', '', text)
        # Removing RT and name of the person who has tweeted
#     text=text.split()
    return text#, negate_flag

In [8]:
def get_negate_flag(text):
    # Remove Negate Word
    return True if [word for word in text.split() if word in NEGATE] else False

In [9]:
import nltk
from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/chinmayivijayendra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
result_df = df[['created_at', 'text']]
result_df

,created_at,text
0,2020-03-06 17:22:02+00:00,RT @carla_denyer: This week has been a particu...
1,2020-03-06 17:22:01+00:00,RT @foeeurope: The kids: this is a #ClimateEme...
2,2020-03-06 17:21:58+00:00,RT @dpcarrington: Just how wet and warm the UK...
3,2020-03-06 17:21:57+00:00,RT @Dan1elGallardo: Ojalá @LaLiga apoye a @gre...
4,2020-03-06 17:21:47+00:00,RT @Green_DannyB: Its worth restating that the...
...,...,...
89345,2020-04-12 18:36:24+00:00,"RT @allafrica: When Biodiversity Fails, Human ..."
89346,2020-04-12 18:21:36+00:00,"RT @Jackthelad1947: After the Coronavirus, Two..."
89347,2020-04-12 17:36:06+00:00,To address #ClimateEmergency we need a #Regene...
89348,2020-04-12 17:34:48+00:00,@DoctorVive @evanlweber This is the most impor...


In [11]:
result_df['cleaned_text'] = result_df['text'].apply(data_clean)
result_df

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,created_at,text,cleaned_text
0,2020-03-06 17:22:02+00:00,RT @carla_denyer: This week has been a particu...,week particularly bad week bristol tackling cl...
1,2020-03-06 17:22:01+00:00,RT @foeeurope: The kids: this is a #ClimateEme...,kids climateemergency keep fossil fuels ground...
2,2020-03-06 17:21:58+00:00,RT @dpcarrington: Just how wet and warm the UK...,wet warm uk february crazyvia metoffice carbon...
3,2020-03-06 17:21:57+00:00,RT @Dan1elGallardo: Ojalá @LaLiga apoye a @gre...,ojal laliga apoye greenpeaceesp en esto los gr...
4,2020-03-06 17:21:47+00:00,RT @Green_DannyB: Its worth restating that the...,worth restating climatestrike happening west t...
...,...,...,...
89345,2020-04-12 18:36:24+00:00,"RT @allafrica: When Biodiversity Fails, Human ...",biodiversity fails human health line africa b...
89346,2020-04-12 18:21:36+00:00,"RT @Jackthelad1947: After the Coronavirus, Two...",coronavirus two sharply divergent paths climat...
89347,2020-04-12 17:36:06+00:00,To address #ClimateEmergency we need a #Regene...,address climateemergency need regenerativeecon...
89348,2020-04-12 17:34:48+00:00,@DoctorVive @evanlweber This is the most impor...,doctorvive evanlweber important election ever ...


In [12]:
result_df['negate_flag'] = result_df.cleaned_text.apply(get_negate_flag)
result_df

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,created_at,text,cleaned_text,negate_flag
0,2020-03-06 17:22:02+00:00,RT @carla_denyer: This week has been a particu...,week particularly bad week bristol tackling cl...,False
1,2020-03-06 17:22:01+00:00,RT @foeeurope: The kids: this is a #ClimateEme...,kids climateemergency keep fossil fuels ground...,False
2,2020-03-06 17:21:58+00:00,RT @dpcarrington: Just how wet and warm the UK...,wet warm uk february crazyvia metoffice carbon...,False
3,2020-03-06 17:21:57+00:00,RT @Dan1elGallardo: Ojalá @LaLiga apoye a @gre...,ojal laliga apoye greenpeaceesp en esto los gr...,False
4,2020-03-06 17:21:47+00:00,RT @Green_DannyB: Its worth restating that the...,worth restating climatestrike happening west t...,False
...,...,...,...,...
89345,2020-04-12 18:36:24+00:00,"RT @allafrica: When Biodiversity Fails, Human ...",biodiversity fails human health line africa b...,False
89346,2020-04-12 18:21:36+00:00,"RT @Jackthelad1947: After the Coronavirus, Two...",coronavirus two sharply divergent paths climat...,False
89347,2020-04-12 17:36:06+00:00,To address #ClimateEmergency we need a #Regene...,address climateemergency need regenerativeecon...,False
89348,2020-04-12 17:34:48+00:00,@DoctorVive @evanlweber This is the most impor...,doctorvive evanlweber important election ever ...,False


In [13]:
from sklearn.cluster import KMeans
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin.gz', binary=True, limit=100000)

In [14]:
cleaned_words = [text.split() for text in result_df.cleaned_text]

word_dict = {}

for words in cleaned_words:
    for word in words:
        if(word in model.vocab):
            word_dict[word] = model[word]

In [15]:
num_of_clusters = len(word_dict)//3

kmeans = KMeans(n_clusters=num_of_clusters, random_state=42)
km = kmeans.fit(list(word_dict.values()))

In [16]:
word_centroid_dict = dict(zip(list(word_dict.keys()), list(kmeans.fit_predict(list(word_dict.values())))))

In [17]:
from sklearn.metrics import pairwise_distances_argmin_min

centroid_words_dict = {}
centroid_and_word = {}

nearest, nearest_array = pairwise_distances_argmin_min(km.cluster_centers_, list(word_dict.values()))

for item in range(num_of_clusters):
    word_vector = list(word_dict.values())[nearest[item]]
    centroid_words_dict[item] = model.most_similar(positive=[word_vector], topn=1)[0][0]

In [18]:
from nltk import sent_tokenize

def replace_words(text):
    temp_text = []
    for word in text.split():
        predicted_word = centroid_words_dict.get(word_centroid_dict.get(word))
        temp_text.append(predicted_word) if predicted_word else temp_text.append(word)
    return " ".join(temp_text)

In [19]:
result_df['word_embeding_text'] = result_df.cleaned_text.apply(replace_words)
result_df

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,created_at,text,cleaned_text,negate_flag,word_embeding_text
0,2020-03-06 17:22:02+00:00,RT @carla_denyer: This week has been a particu...,week particularly bad week bristol tackling cl...,False,day particularly crummy day bristol tackling c...
1,2020-03-06 17:22:01+00:00,RT @foeeurope: The kids: this is a #ClimateEme...,kids climateemergency keep fossil fuels ground...,False,children climateemergency keep fossil fueling ...
2,2020-03-06 17:21:58+00:00,RT @dpcarrington: Just how wet and warm the UK...,wet warm uk february crazyvia metoffice carbon...,False,dry warmed uk february crazyvia metoffice carb...
3,2020-03-06 17:21:57+00:00,RT @Dan1elGallardo: Ojalá @LaLiga apoye a @gre...,ojal laliga apoye greenpeaceesp en esto los gr...,False,ojal laliga apoye greenpeaceesp en esto los gr...
4,2020-03-06 17:21:47+00:00,RT @Green_DannyB: Its worth restating that the...,worth restating climatestrike happening west t...,False,worth restating climatestrike happens east cer...
...,...,...,...,...,...
89345,2020-04-12 18:36:24+00:00,"RT @allafrica: When Biodiversity Fails, Human ...",biodiversity fails human health line africa b...,False,ecosystems failed natural health line africa e...
89346,2020-04-12 18:21:36+00:00,"RT @Jackthelad1947: After the Coronavirus, Two...",coronavirus two sharply divergent paths climat...,False,coronavirus three slightly overlapping path cl...
89347,2020-04-12 17:36:06+00:00,To address #ClimateEmergency we need a #Regene...,address climateemergency need regenerativeecon...,False,addressed climateemergency need regenerativeec...
89348,2020-04-12 17:34:48+00:00,@DoctorVive @evanlweber This is the most impor...,doctorvive evanlweber important election ever ...,False,doctorvive evanlweber vital election perhaps a...


In [20]:
def get_hypernyms(text):
    temp_word = []
    for word in text.split():
        syn = wn.synsets(word)
        if syn:
            hyp = syn[0].hypernyms()
            if hyp:
                temp_word.append(syn[0].hypernyms()[0].lemma_names()[0])
    return " ".join(temp_word)

In [21]:
result_df['hypernym_text'] = result_df.cleaned_text.apply(get_hypernyms)
result_df

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,created_at,text,cleaned_text,negate_flag,word_embeding_text,hypernym_text
0,2020-03-06 17:22:02+00:00,RT @carla_denyer: This week has been a particu...,week particularly bad week bristol tackling cl...,False,day particularly crummy day bristol tackling c...,time_period quality time_period confront class...
1,2020-03-06 17:22:01+00:00,RT @foeeurope: The kids: this is a #ClimateEme...,kids climateemergency keep fossil fuels ground...,False,children climateemergency keep fossil fueling ...,juvenile resource oldster substance object abs...
2,2020-03-06 17:21:58+00:00,RT @dpcarrington: Just how wet and warm the UK...,wet warm uk february crazyvia metoffice carbon...,False,dry warmed uk february crazyvia metoffice carb...,wetness change Gregorian_calendar_month
3,2020-03-06 17:21:57+00:00,RT @Dan1elGallardo: Ojalá @LaLiga apoye a @gre...,ojal laliga apoye greenpeaceesp en esto los gr...,False,ojal laliga apoye greenpeaceesp en esto los gr...,linear_unit baseball_team argument angular_pos...
4,2020-03-06 17:21:47+00:00,RT @Green_DannyB: Its worth restating that the...,worth restating climatestrike happening west t...,False,worth restating climatestrike happens east cer...,indefinite_quantity tell event
...,...,...,...,...,...,...
89345,2020-04-12 18:36:24+00:00,"RT @allafrica: When Biodiversity Fails, Human ...",biodiversity fails human health line africa b...,False,ecosystems failed natural health line africa e...,diverseness hominid wellbeing formation divers...
89346,2020-04-12 18:21:36+00:00,"RT @Jackthelad1947: After the Coronavirus, Two...",coronavirus two sharply divergent paths climat...,False,coronavirus three slightly overlapping path cl...,digit course environmental_condition
89347,2020-04-12 17:36:06+00:00,To address #ClimateEmergency we need a #Regene...,address climateemergency need regenerativeecon...,False,addressed climateemergency need regenerativeec...,code condition bodily_property condition make
89348,2020-04-12 17:34:48+00:00,@DoctorVive @evanlweber This is the most impor...,doctorvive evanlweber important election ever ...,False,doctorvive evanlweber vital election perhaps a...,vote


In [22]:
def score_cal(text):
#     print(text)
    temp_pos_res = [] # creating empty positive score list
    temp_neg_res = [] # creating empty negative score list
    for word in text.split(): # for each word in embedded text
        res = wn.synsets(word) # find the synsets and store it in avariable
        if res: # if synsets are found
            pos_avg_word = 0 
            neg_avg_word = 0
            count = 0
            for syn in res: #for each synset in synsets
                score = swn.senti_synset(syn.name()) #calculate the sentiment score 
                if score.pos_score() > 0 or score.neg_score() > 0: # if positive or negative sentiment is present
                    pos_avg_word += score.pos_score() # assign positive score
                    neg_avg_word += score.neg_score() # assign negative score
                    count += 1 # increment the count for positive or negative synsets 
            if count > 0:  # if positive or negative score is set  
                temp_pos_res.append(pos_avg_word/count) # calculate the average positive score with respect to synset
                temp_neg_res.append(neg_avg_word/count) # calculate the average  negative score 
    if temp_pos_res and temp_neg_res: # if positive or negative score found in syssets
        pos_sc = sum(temp_pos_res)/len(temp_pos_res) # average with respect to positive synsets score
        neg_sc = sum(temp_neg_res)/len(temp_neg_res) # average with respect to negative synsets score
    else:
        pos_sc = 0
        neg_sc = 0
    score = pos_sc - neg_sc # calculate the sentiment score by subtracting negative score from positive score
    if score > 0: # if the calculated sentiment score is poisitive 
        return 1 # return positive sentiment
    elif score < 0: #if the calculated sentiment score is negative
        return -1 #  return negative sentiment
    else:
        return 0 # else return netural

In [23]:
result_df['cleaned_text_score'] = result_df.cleaned_text.apply(score_cal)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
result_df['embeded_text_score'] = result_df.word_embeding_text.apply(score_cal)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
result_df['hypernym_score'] = result_df.hypernym_text.apply(score_cal)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [26]:
def negate_result_calculator(negate_flag, result):
    result_list = []
    for nf, res in zip(negate_flag, result):
        if nf:
            res = res * (-1)
        result_list.append(res)
    return result_list

In [27]:
for field in ['cleaned_text_score', 'embeded_text_score', 'hypernym_score']:
    result_df[field] = negate_result_calculator(result_df['negate_flag'], result_df[field])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
result_df

,created_at,text,cleaned_text,negate_flag,word_embeding_text,hypernym_text,cleaned_text_score,embeded_text_score,hypernym_score
0,2020-03-06 17:22:02+00:00,RT @carla_denyer: This week has been a particu...,week particularly bad week bristol tackling cl...,False,day particularly crummy day bristol tackling c...,time_period quality time_period confront class...,-1,-1,1
1,2020-03-06 17:22:01+00:00,RT @foeeurope: The kids: this is a #ClimateEme...,kids climateemergency keep fossil fuels ground...,False,children climateemergency keep fossil fueling ...,juvenile resource oldster substance object abs...,-1,-1,1
2,2020-03-06 17:21:58+00:00,RT @dpcarrington: Just how wet and warm the UK...,wet warm uk february crazyvia metoffice carbon...,False,dry warmed uk february crazyvia metoffice carb...,wetness change Gregorian_calendar_month,-1,-1,1
3,2020-03-06 17:21:57+00:00,RT @Dan1elGallardo: Ojalá @LaLiga apoye a @gre...,ojal laliga apoye greenpeaceesp en esto los gr...,False,ojal laliga apoye greenpeaceesp en esto los gr...,linear_unit baseball_team argument angular_pos...,0,1,1
4,2020-03-06 17:21:47+00:00,RT @Green_DannyB: Its worth restating that the...,worth restating climatestrike happening west t...,False,worth restating climatestrike happens east cer...,indefinite_quantity tell event,1,1,-1
...,...,...,...,...,...,...,...,...,...
89345,2020-04-12 18:36:24+00:00,"RT @allafrica: When Biodiversity Fails, Human ...",biodiversity fails human health line africa b...,False,ecosystems failed natural health line africa e...,diverseness hominid wellbeing formation divers...,1,-1,1
89346,2020-04-12 18:21:36+00:00,"RT @Jackthelad1947: After the Coronavirus, Two...",coronavirus two sharply divergent paths climat...,False,coronavirus three slightly overlapping path cl...,digit course environmental_condition,1,1,-1
89347,2020-04-12 17:36:06+00:00,To address #ClimateEmergency we need a #Regene...,address climateemergency need regenerativeecon...,False,addressed climateemergency need regenerativeec...,code condition bodily_property condition make,1,1,1
89348,2020-04-12 17:34:48+00:00,@DoctorVive @evanlweber This is the most impor...,doctorvive evanlweber important election ever ...,False,doctorvive evanlweber vital election perhaps a...,vote,1,1,1


In [29]:
result_df = result_df.loc[:, result_df.columns != 'negate_flag']
result_df

,created_at,text,cleaned_text,word_embeding_text,hypernym_text,cleaned_text_score,embeded_text_score,hypernym_score
0,2020-03-06 17:22:02+00:00,RT @carla_denyer: This week has been a particu...,week particularly bad week bristol tackling cl...,day particularly crummy day bristol tackling c...,time_period quality time_period confront class...,-1,-1,1
1,2020-03-06 17:22:01+00:00,RT @foeeurope: The kids: this is a #ClimateEme...,kids climateemergency keep fossil fuels ground...,children climateemergency keep fossil fueling ...,juvenile resource oldster substance object abs...,-1,-1,1
2,2020-03-06 17:21:58+00:00,RT @dpcarrington: Just how wet and warm the UK...,wet warm uk february crazyvia metoffice carbon...,dry warmed uk february crazyvia metoffice carb...,wetness change Gregorian_calendar_month,-1,-1,1
3,2020-03-06 17:21:57+00:00,RT @Dan1elGallardo: Ojalá @LaLiga apoye a @gre...,ojal laliga apoye greenpeaceesp en esto los gr...,ojal laliga apoye greenpeaceesp en esto los gr...,linear_unit baseball_team argument angular_pos...,0,1,1
4,2020-03-06 17:21:47+00:00,RT @Green_DannyB: Its worth restating that the...,worth restating climatestrike happening west t...,worth restating climatestrike happens east cer...,indefinite_quantity tell event,1,1,-1
...,...,...,...,...,...,...,...,...
89345,2020-04-12 18:36:24+00:00,"RT @allafrica: When Biodiversity Fails, Human ...",biodiversity fails human health line africa b...,ecosystems failed natural health line africa e...,diverseness hominid wellbeing formation divers...,1,-1,1
89346,2020-04-12 18:21:36+00:00,"RT @Jackthelad1947: After the Coronavirus, Two...",coronavirus two sharply divergent paths climat...,coronavirus three slightly overlapping path cl...,digit course environmental_condition,1,1,-1
89347,2020-04-12 17:36:06+00:00,To address #ClimateEmergency we need a #Regene...,address climateemergency need regenerativeecon...,addressed climateemergency need regenerativeec...,code condition bodily_property condition make,1,1,1
89348,2020-04-12 17:34:48+00:00,@DoctorVive @evanlweber This is the most impor...,doctorvive evanlweber important election ever ...,doctorvive evanlweber vital election perhaps a...,vote,1,1,1


In [30]:
# result_df.to_csv('Final_Result.csv', index=False)
# result_df.to_json('climate_emergency_output_file.json')

In [31]:
#Grouping Categories

In [32]:
new_words =['weather','pollution','policy','action','environment','plastic','initiatives','recycle','earth','people']
# Add the coosen words of category to the variable new_words.
new_word_dict = {}

for new_word in new_words:
    if(new_word in model.vocab): # If the choosen word category in vocab the add to variable new_word_dict.
        new_word_dict[new_word] = model[new_word]

In [33]:
from sklearn.metrics import pairwise_distances_argmin_min


def get_nearest_word(words):
    result = []
    for word in words: # for all the words in word_dict
        temp_list = []
        for i in range(len(new_word_dict)):
            temp_list.append(word)

        nearest, dist = pairwise_distances_argmin_min(list(new_word_dict.values()), temp_list)
        # calculate a minimum distance between each category word and a word in word_dict.
    
        result.append(list(dist).index(dist.min())) # consider the minimum distance category for that paricular wordl.
    return result

In [34]:
word_centroid_dict = dict(zip(list(word_dict.keys()), get_nearest_word(list(word_dict.values()))))

In [35]:
centroid_words_dict = {}
for index in range(len(new_word_dict)):
    centroid_words_dict[index] = list(new_word_dict.keys())[index]
centroid_words_dict

{0: 'weather',
 1: 'pollution',
 2: 'policy',
 3: 'action',
 4: 'environment',
 5: 'plastic',
 6: 'initiatives',
 7: 'recycle',
 8: 'earth',
 9: 'people'}

In [36]:
new_res_df = pd.DataFrame({"Word": list(word_centroid_dict.keys()), "Centroid Word": list(word_centroid_dict.values())})


In [37]:
new_res_df['Centroid Word'] = new_res_df['Centroid Word'].apply(lambda x: centroid_words_dict[x])


In [38]:
# new_res_df.to_csv("/Users/chinmayivijayendra/Desktop/saveThePlanet.csv")